# 1. Setup

## 1.1 Load Libraries

In [1]:
install.packages("webshot2")


Os pacotes bin'arios baixados est~ao em
	/var/folders/ln/zzm22ch17c7522fvnzbc0d4c0000gn/T//RtmpujQhr1/downloaded_packages


In [2]:
library(tidyverse) # Includes ggplot2, dplyr, tidyr, readr, purrr, tibble, stringr, forcats
library(tidylog) # For better logging
library(plm) # For panel data analysis
library(sandwich) # For robust standard errors
library(lmtest) # For robust standard errors
library(stargazer) # For regression tables
library(jtools) # Load jtools
library(clubSandwich) # To clusterize SEs
library(fixest) # To estiamate clustered robust SEs
library(multiwayvcov) # To estimate the vcov on TWFE
library(etwfe) # To estimate a TWFE
library(gt) # To make tables
library(styler) # To stylize code
library(lfe) # alternative to plm
library(modelsummary) # To make beautiful results tables
library(kableExtra)
library(webshot2)
library(gt)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Anexando pacote: 'tidylog'


Os seguintes objetos s~ao mascarados por 'package:dplyr':

    add_count, add_tally, anti_join, count, distinct, distinct_all,
    distinct_at, distinct_if, filter, filter_all, filter_at, filter_if,
    full_join, group_by, group_by_all, group_by_at, group_by_if,
    inner_join, left_join, mutate, mutate_all, mutate_at, mutate_if,
    relocate, rename, rename_all, rename_at, rename_if, rename_with,
    right_join, sample_frac, sample_n, selec

## 1.2 Set wd()

In [3]:
desired_path <- "/Users/vhpf/Library/CloudStorage/OneDrive-Personal/Mestrado/pantanal_project" # Change to the folder of your project
setwd(desired_path)

## 1.3 Other Adjustments

In [4]:
Sys.setenv(LANG = "en_US.UTF-8") #Set language to English
options(scipen = 999) # Get rid of scientific notation
memory.limit(size = 32000) # Increase the memory limit to 32 GB

Warning message:
"'memory.limit()' is Windows-specific"


[1] Inf

## 1.4 Function to Summarize the Results

In [5]:
# Function to test for significance, compute confidence intervals, and summarize results
summarize_model_results <- function(model, vcov) {
    # Step 3: Test for significance
    results <- coeftest(model, vcov = vcov)
    
    # Step 4: 95% confidence interval
    conf_intervals <- confint(model, vcov. = vcov)
    
    # Step 5: Results
    results_df <- tidy(results)
    summary_table <- results_df %>%
        mutate_if(is.numeric, round, 4) %>% 
        mutate(
            ci95_lower = conf_intervals[, 1],
            ci95_upper = conf_intervals[, 2],
            significance = case_when(
                p.value < 0.001 ~ "***",
                p.value < 0.01 ~ "**",
                p.value < 0.05 ~ "*",
                p.value < 0.1 ~ ".",
                TRUE ~ "No"
            )
        )

    # Step 6: Drop the factor(year) rows
    summary_table <- summary_table %>%
        filter(!term %in% paste0("factor(year)", c(1986, 2023)))

    # Display the updated summary table
    return(summary_table)
}

## 1.5 Function to Generically Estimate the FD Model

In [6]:
get_summary_table <- function(y, x, dataset, cluster) {
    # Combine the items in x into a single formula string
    x_formula <- paste(x, collapse = " + ")
    
    # Create the formula string
    formula_str <- paste(y, "~", x_formula, "+ factor(year)")
    formula <- as.formula(formula_str)
    
    # 1. Estimate the FE model
    model <- plm(
        formula = formula, # Regress water on upstream area and dummy for year
        data = dataset, # Dataset used
        index = c("wts_cd_pfafstetterbasin", "year"), # i = wts_cd_pfafstetterbasin, t = year
        model = "fd" # FD model
    )

    # 2. Estimate the vcov matrix
    vcov <- plm::vcovHC(
        model, 
        method = "arellano", # use Arellano (1987) - errors have heteroskedasticity and serial (cross–sectional) correlation
        type = "HC1", # Use HC1 estimator to adjust for this
        cluster = "group", # Within group correlation
        group = dataset[[cluster]] # analogous to Stata's cluster(clusterid)
    )

    summary_table <- summarize_model_results(model, vcov) # Get the CIs and make the results table
    return(summary_table)
}

## 1.6 Function that Returns the Estimate and the Vcov of the Model

In [7]:
fd_model <- function(y, x, dataset, cluster) {
    # Combine the items in x into a single formula string
    x_formula <- paste(x, collapse = " + ")
    
    # Create the formula string
    formula_str <- paste(y, "~", x_formula, "+ factor(year)")
    formula <- as.formula(formula_str)
    
    # 1. Estimate the FE model
    model <- plm(
        formula = formula, # Regress water on upstream area and dummy for year
        data = dataset, # Dataset used
        index = c("wts_cd_pfafstetterbasin", "year"), # i = wts_cd_pfafstetterbasin, t = year
        model = "fd" # FD model
    )

    # 2. Estimate the vcov matrix
    vcov <- plm::vcovHC(
        model, 
        method = "arellano", # use Arellano (1987) - errors have heteroskedasticity and serial (cross–sectional) correlation
        type = "HC1", # Use HC1 estimator to adjust for this
        cluster = "group", # Within group correlation
        group = dataset[[cluster]]
    )

    results <- c(model, vcov)
    return(results)
}

# 2. Load Data

In [8]:
scale_factor = 1

In [9]:
panel_path <- "output/data/panel.csv" #Path to the panel data
panel.R <- read_csv(panel_path)
panel <- panel.R %>% tibble()
panel <- panel %>% mutate(
    water_basin_km2 = water_basin_km2*scale_factor,
    wetland_basin_km2 = wetland_basin_km2*scale_factor,
    fire_basin_km2 = fire_basin_km2*scale_factor,
    antropic_upstream_km2 = antropic_upstream_km2*scale_factor,
    antropic_downstream_km2 = antropic_downstream_km2*scale_factor,

    log_water_basin_km2 = asinh(water_basin_km2),
    log_wetland_basin_km2 = asinh(wetland_basin_km2),
    log_fire_basin_km2 = asinh(fire_basin_km2),
    log_antropic_upstream_km2 = asinh(antropic_upstream_km2), 
    log_antropic_downstream_km2 = asinh(antropic_downstream_km2),
    log_upstream_rainfall = asinh(upstream_rainfall)
    )
panel <- data.frame(panel)

Rows: 73047 Columns: 21
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (21): wts_cd_pfafstetterbasin, year, level5, level4, level3, water_basin...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
mutate: new variable 'log_water_basin_km2' (double) with 17,215 unique values and 0% NA
        new variable 'log_wetland_basin_km2' (double) with 27,802 unique values and 0% NA
        new variable 'log_fire_basin_km2' (double) with 16,376 unique values and 0% NA
        new variable 'log_antropic_upstream_km2' (double) with 27,429 unique values and 0% NA
        new variable 'log_antropic_downstream_km2' (double) with 22,731 unique values and 0% NA
        new variable 'log_upstream_rainfall' (double) with 32,990 unique values and 0% NA


## 2.1 Change in Water Area caused by Upstream Deforestation

In [14]:
panel_change <- panel %>% filter(in_biome == 1) %>% group_by(year) %>% 
    summarize(
        sum_water = sum(water_basin_km2),
        sum_wetland = sum(wetland_basin_km2),
        sum_fire = sum(fire_basin_km2),
        sum_anthropic_upstream = sum(antropic_upstream_km2),
        total_area = sum(area_basin_km2)) %>% 
    mutate(share_water = (sum_water/total_area)*100,
           share_fire = (sum_fire/total_area)*100)
panel_change

filter: removed 32,019 rows (44%), 41,028 rows remaining
group_by: one grouping variable (year)


summarize: now 39 rows and 6 columns, ungrouped
mutate: new variable 'share_water' (double) with 39 unique values and 0% NA
        new variable 'share_fire' (double) with 39 unique values and 0% NA


year,sum_water,sum_wetland,sum_fire,sum_anthropic_upstream,total_area,share_water,share_fire
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1985,32402.905,38417.167,6869.1825,5143104,152499.7,21.247849,4.5043910
1986,12424.463,27607.738,9821.4417,5450715,152499.8,8.147197,6.4402962
1987,15874.669,39521.054,4246.9839,5649423,152499.8,10.409634,2.7849114
1988,31927.548,36719.090,10802.2689,5834350,152499.9,20.936111,7.0834597
1989,25323.854,43587.371,1863.8919,6068136,152500.1,16.605797,1.2222236
1990,12711.450,28565.913,4997.3139,6264681,152500.1,8.335372,3.2769253
1991,20662.278,34799.982,9287.4753,6443436,152500.1,13.549025,6.0901432
1992,18398.757,22653.642,2560.8015,6669456,152500.2,12.064742,1.6792118
1993,17732.521,30271.841,15212.6802,6854327,152500.3,11.627859,9.9755075


# 3. Main Model: FD - Water

$$y_{it} = \mu_i + \beta \cdot X_{it} + \epsilon_{it}$$
Take the first diffence: $\Delta y_{it} = y_{it} - y_{i,t-1}$
$$\Delta y_{it} = \beta \cdot \Delta X_{it} + \epsilon_{it}$$
But we estimate with the intercept:
$$\Delta y_{it} = \beta_0 + \beta \cdot \Delta X_{it} + \epsilon_{it}$$

## 3.1 Model 1: FD log(water area) on log(upstream area)

In [27]:
summary_table1 <- get_summary_table(
    y = 'log_water_basin_km2', 
    x = c('log_antropic_upstream_km2'),
    dataset = panel,
    cluster = 'level5')

mutate_if: changed 39 values (100%) of 'estimate' (0 new NAs)
           changed 39 values (100%) of 'std.error' (0 new NAs)
           changed 39 values (100%) of 'statistic' (0 new NAs)
           changed 39 values (100%) of 'p.value' (0 new NAs)
mutate: new variable 'ci95_lower' (double) with 39 unique values and 0% NA
        new variable 'ci95_upper' (double) with 39 unique values and 0% NA
        new variable 'significance' (character) with 4 unique values and 0% NA
filter: removed one row (3%), 38 rows remaining


## 3.2 Model 2: FD log(water area) on log(upstream area) + log(upstream rainfall)

In [28]:
summary_table2 <- get_summary_table(
    y = 'log_water_basin_km2', 
    x = c('log_antropic_upstream_km2', 'log_upstream_rainfall'),
    dataset = panel,
    cluster = 'level5')

mutate_if: changed 40 values (100%) of 'estimate' (0 new NAs)
           changed 40 values (100%) of 'std.error' (0 new NAs)
           changed 40 values (100%) of 'statistic' (0 new NAs)
           changed 40 values (100%) of 'p.value' (0 new NAs)
mutate: new variable 'ci95_lower' (double) with 40 unique values and 0% NA
        new variable 'ci95_upper' (double) with 40 unique values and 0% NA
        new variable 'significance' (character) with 4 unique values and 0% NA
filter: removed one row (2%), 39 rows remaining


## 3.3 Model 3: FD log(water area) on log(downstream area)

In [29]:
summary_table3 <- get_summary_table(
    y = 'log_water_basin_km2', 
    x = c('log_antropic_downstream_km2'),
    dataset = panel,
    cluster = 'level5')

mutate_if: changed 39 values (100%) of 'estimate' (0 new NAs)
           changed 39 values (100%) of 'std.error' (0 new NAs)
           changed 39 values (100%) of 'statistic' (0 new NAs)
           changed 39 values (100%) of 'p.value' (0 new NAs)
mutate: new variable 'ci95_lower' (double) with 39 unique values and 0% NA
        new variable 'ci95_upper' (double) with 39 unique values and 0% NA
        new variable 'significance' (character) with 5 unique values and 0% NA
filter: removed one row (3%), 38 rows remaining


# 4. Main Model: FD - Fire

$$y_{it} = \mu_i + \beta \cdot X_{it} + \epsilon_{it}$$
Take the first diffence: $\Delta y_{it} = y_{it} - y_{i,t-1}$
$$\Delta y_{it} = \beta \cdot \Delta X_{it} + \epsilon_{it}$$
But we estimate with the intercept:
$$\Delta y_{it} = \beta_0 + \beta \cdot \Delta X_{it} + \epsilon_{it}$$

## 4.1 Model 4: FD log(fire area) on log(upstream area)

In [30]:
summary_table4 <- get_summary_table(
    y = 'log_fire_basin_km2', 
    x = c('log_antropic_upstream_km2'),
    dataset = panel,
    cluster = 'level5')

mutate_if: changed 39 values (100%) of 'estimate' (0 new NAs)
           changed 39 values (100%) of 'std.error' (0 new NAs)
           changed 39 values (100%) of 'statistic' (0 new NAs)
           changed 38 values (97%) of 'p.value' (0 new NAs)
mutate: new variable 'ci95_lower' (double) with 39 unique values and 0% NA
        new variable 'ci95_upper' (double) with 39 unique values and 0% NA
        new variable 'significance' (character) with 4 unique values and 0% NA
filter: removed one row (3%), 38 rows remaining


## 4.2 Model 5: FD log(fire area) on log(upstream area) + log(upstream rainfall)

In [31]:
summary_table5 <- get_summary_table(
    y = 'log_fire_basin_km2', 
    x = c('log_antropic_upstream_km2', 'log_upstream_rainfall'),
    dataset = panel,
    cluster = 'level5')

mutate_if: changed 40 values (100%) of 'estimate' (0 new NAs)
           changed 40 values (100%) of 'std.error' (0 new NAs)
           changed 40 values (100%) of 'statistic' (0 new NAs)
           changed 40 values (100%) of 'p.value' (0 new NAs)
mutate: new variable 'ci95_lower' (double) with 40 unique values and 0% NA
        new variable 'ci95_upper' (double) with 40 unique values and 0% NA
        new variable 'significance' (character) with 5 unique values and 0% NA
filter: removed one row (2%), 39 rows remaining


# 5. Table with the Main Results

In [32]:
## Function to run generic FD models to better use Stargazzer later
run_regressions <- function(regressors, data, index, model_type = "fd", cluster_level) {
    # Lists to append the objects
    models <- list()
    vcov_list <- list()
    
    # Run the various models and get the point estimates and the vcovs
    for (i in 1:length(regressors)) {
        models[[i]] <- plm(
            formula = regressors[[i]], # Use the corresponding formula
            data = data, # Adjust this if data subsets are needed
            index = index,
            model = model_type
        )
        
        vcov_list[[i]] <- plm::vcovHC(
            models[[i]],
            method = "arellano",
            type = "HC1",
            cluster = "group",
            group = data[[cluster_level]]
        )
    }
    
    return(list(models = models, vcov_list = vcov_list))
}


In [33]:
# 1. Set the multiple formulas
regressors <- list(
  formula1 = log_wetland_basin_km2 ~ log_antropic_upstream_km2,
  formula2 = log_wetland_basin_km2 ~ log_antropic_upstream_km2 + factor(year),
  formula3 = log_fire_basin_km2 ~ log_antropic_upstream_km2,
  formula4 = log_fire_basin_km2 ~ log_antropic_upstream_km2 + factor(year)
)

# 2. Lists to append the objects
models <- list()
vcov_list <- list()

In [34]:
## Model 1
i = 1
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

## Model 2
i = 2
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

## Model 3
i = 3
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

## Model 4
i = 4
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

In [36]:
# Create a sequence of years from 1986 to 2023
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Use the list in the stargazer function
stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]], # Include all models
    type = "text", # Output as LaTeX
    se = lapply(vcov_list, function(x) sqrt(diag(x))), # Extract robust SEs
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE, # Do not include default dependent variable labels
    dep.var.caption = "", # Remove the "Dependent variable:" text
    column.labels = c("log(Water Area)", "log(Fire Area)"), # Specify dependent variables above model numbers
    column.separate = c(2, 2), # Group columns by dependent variable
    omit.stat = c("f", "ser", "rsq"), # Omit F-stat and SER if needed
    multicolumn = TRUE,
    add.lines = list(
        c("Cluster", "Level 5", "Level 5", "Level 5", "Level 5"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "No", "Yes", "No", "Yes"),
        c("\\hline \\noalign{\\vskip 0.1ex}") # Add a half-height line for better visualization
    ),
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    notes = "Standard errors are shown in parentheses.", # Add notes
    out = "output_table.tex" # Specify the output file path
)


                           log(Water Area)        log(Fire Area)    
                            (1)        (2)        (3)        (4)    
--------------------------------------------------------------------
(Intercept)               -0.001*    -0.001*   -0.032***  -0.034*** 
                          (0.001)    (0.001)    (0.002)    (0.002)  
log(Anthropic Upstream)  -0.235***  -0.234***   0.353***   0.529*** 
                          (0.061)    (0.059)    (0.123)    (0.106)  
--------------------------------------------------------------------
Cluster                   Level 5    Level 5    Level 5    Level 5  
Entity FE                   Yes        Yes        Yes        Yes    
Year Dummy                  No         Yes         No        Yes    
0.1ex                                                               
Observations              71,174      71,174     71,174     71,174  
Adjusted R2               0.0003      0.133      0.0002     0.135   
Note:                            

# 6. Robustness Checks

In [270]:
# 1. Set the multiple formulas
regressors <- list(
  formula1 = log_water_basin_km2 ~ log_antropic_downstream_km2 + factor(year), # Test 1: deforestation downstream
  formula2 = log_water_basin_km2 ~ log_antropic_upstream_km2 + log_upstream_rainfall + factor(year), # Test 2: Rainfall
  formula3 = log_water_basin_km2 ~ log_antropic_upstream_km2 + factor(year), # Test 3: no cluster
  formula4 = log_fire_basin_km2 ~ log_antropic_upstream_km2 + log_upstream_rainfall + factor(year), # Test 4: test 2 with y=fire
  formula5 = log_fire_basin_km2 ~ log_antropic_upstream_km2 + factor(year) # Test 5: test 3 with y=fire
)

# 2. Lists to append the objects
models <- list()
vcov_list <- list()

In [271]:
## Model 1
i = 1
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

## Model 2
i = 2
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

## Model 3
i = 3
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1")

## Model 4
i = 4
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel$level5)

## Model 5
i = 5
models[[i]] <- plm(
    formula = regressors[[i]], data = panel, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1")

In [272]:
# Create a sequence of years from 1986 to 2023
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Use the list in the stargazer function
stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]], models[[5]],  # Include all models
    type = "latex", # Output as LaTeX
    se = lapply(vcov_list, function(x) sqrt(diag(x))), # Extract robust SEs
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE, # Do not include default dependent variable labels
    dep.var.caption = "", # Remove the "Dependent variable:" text
    column.labels = c("log(Water Area)", "log(Fire Area)"), # Specify dependent variables above model numbers
    column.separate = c(3, 2), # Group columns by dependent variable
    omit.stat = c("f", "ser", "rsq"), # Omit F-stat and SER if needed
    multicolumn = TRUE,
    add.lines = list(
        c("Cluster", "Level 5", "Level 5", "No", "Level 5", "No"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "Yes", "Yes", "Yes", "Yes", "Yes"),
        c("\\hline \\noalign{\\vskip 0.1ex}") # Add a half-height line for better visualization
    ),
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    covariate.labels = c("(Intercept)", "log(Anthropic Downstream)", "log(Anthropic Upstream)", "log(Upstream Rainfall)"),
    intercept.bottom = FALSE,
    notes = "Standard errors are shown in parentheses.", # Add notes
    out = "output_table.tex" # Specify the output file path
)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Feb 05, 2025 - 11:50:38
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{log(Water Area)} & \multicolumn{2}{c}{log(Fire Area)} \\ 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)} & \multicolumn{1}{c}{(3)} & \multicolumn{1}{c}{(4)} & \multicolumn{1}{c}{(5)}\\ 
\hline \\[-1.8ex] 
 (Intercept) & -0.021^{***} & -0.018^{***} & -0.018^{***} & -0.034^{***} & -0.034^{***} \\ 
  & (0.003) & (0.002) & (0.001) & (0.002) & (0.002) \\ 
  log(Anthropic Downstream) & -0.051 &  &  &  &  \\ 
  & (0.099) &  &  &  &  \\ 
  log(Anthropic Upstream) &  & -0.397^{***} & -0.397^{***} & 0.531^{***} & 0.529^{***} \\ 
  &  & (0.124) & (0.124) & (0.106) & (0.106) 

# 7. Heterogenous Effects

## 7.1 Central Basins Only

In [273]:
panel_central <- panel %>% filter(in_biome == 1)
panel_highlands <- panel %>% filter(in_biome == 0)

filter: removed 32,019 rows (44%), 41,028 rows remaining
filter: removed 41,028 rows (56%), 32,019 rows remaining


In [274]:
# 1. Set the multiple formulas
regressors <- list(
  formula1 = log_water_basin_km2 ~ log_antropic_upstream_km2 + factor(year), #Central
  formula2 = log_water_basin_km2 ~ log_antropic_upstream_km2 + factor(year), #Highlands
  formula3 = log_fire_basin_km2 ~ log_antropic_upstream_km2 + factor(year), # Central
  formula4 = log_fire_basin_km2 ~ log_antropic_upstream_km2 + factor(year) # Highlands
)

# 2. Lists to append the objects
models <- list()
vcov_list <- list()

In [275]:
## Model 1
i = 1
models[[i]] <- plm(
    formula = regressors[[i]], data = panel_central, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel_central$level5)

## Model 2
i = 2
models[[i]] <- plm(
    formula = regressors[[i]], data = panel_highlands, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel_highlands$level5)

## Model 3
i = 3
models[[i]] <- plm(
    formula = regressors[[i]], data = panel_central, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel_central$level5)

## Model 4
i = 4
models[[i]] <- plm(
    formula = regressors[[i]], data = panel_highlands, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd")
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = panel_highlands$level5)

In [276]:
# Create a sequence of years from 1986 to 2023
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Use the list in the stargazer function
stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]],  # Include all models
    type = "latex", # Output as LaTeX
    se = lapply(vcov_list, function(x) sqrt(diag(x))), # Extract robust SEs
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE, # Do not include default dependent variable labels
    dep.var.caption = "", # Remove the "Dependent variable:" text
    column.labels = c("log(Water Area)", "log(Fire Area)"), # Specify dependent variables above model numbers
    column.separate = c(2, 2), # Group columns by dependent variable
    omit.stat = c("f", "ser", "rsq"), # Omit F-stat and SER if needed
    multicolumn = TRUE,
    add.lines = list(
        c('Group', "Biome", "Highlands", "Biome", "Highlands"),
        c("Cluster", "Level 5", "Level 5", "Level 5", "Level 5"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "Yes", "Yes", "Yes", "Yes"),
        c("\\hline \\noalign{\\vskip 0.1ex}") # Add a half-height line for better visualization
    ),
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    notes = "Standard errors are shown in parentheses.", # Add notes
    out = "output_table.tex" # Specify the output file path
)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Feb 05, 2025 - 11:50:43
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{log(Water Area)} & \multicolumn{2}{c}{log(Fire Area)} \\ 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)} & \multicolumn{1}{c}{(3)} & \multicolumn{1}{c}{(4)}\\ 
\hline \\[-1.8ex] 
 (Intercept) & -0.034^{***} & -0.001^{**} & -0.017^{***} & -0.055^{***} \\ 
  & (0.002) & (0.001) & (0.002) & (0.002) \\ 
  log(Anthropic Upstream) & -0.508^{***} & 0.089^{***} & 0.554^{***} & 0.409^{***} \\ 
  & (0.133) & (0.022) & (0.119) & (0.125) \\ 
 \hline \\[-1.8ex] 
Group & Biome & Highlands & Biome & Highlands \\ 
Cluster & Level 5 & Level 5 & Level 5 & Level 5 \\ 
Entity FE & Yes & Yes & Ye

## 7.2 Quintiles of Time

In [277]:
panel_time <- panel %>% mutate(
    subset = case_when(
        year %in% 1985:1994 ~ "Q1",
        year %in% 1995:2004 ~ "Q2",
        year %in% 2005:2014 ~ "Q3",
        year %in% 2015:2023 ~ "Q4",
        TRUE ~ NA_character_ # Default case if year does not match any range
    )
)
panel_time_q1 <- panel_time %>% filter(subset == 'Q1')
panel_time_q2 <- panel_time %>% filter(subset == 'Q2')
panel_time_q3 <- panel_time %>% filter(subset == 'Q3')
panel_time_q4 <- panel_time %>% filter(subset == 'Q4')

mutate: new variable 'subset' (character) with 4 unique values and 0% NA
filter: removed 54,317 rows (74%), 18,730 rows remaining
filter: removed 54,317 rows (74%), 18,730 rows remaining
filter: removed 54,317 rows (74%), 18,730 rows remaining
filter: removed 56,190 rows (77%), 16,857 rows remaining


In [278]:
# 1. Set the multiple formulas
regressors <- list(
  formula_water = log_water_basin_km2 ~ log_antropic_upstream_km2 + factor(year),
  formula_fire = log_fire_basin_km2 ~ log_antropic_upstream_km2 + factor(year)
)

# 2. Lists to append the objects
models <- list()
vcov_list <- list()

### 7.3.1 Water

In [279]:
## Model 1
i = 1
data = panel_time_q1
models[[i]] <- plm(
    formula = regressors[[1]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

## Model 2
i = 2
data = panel_time_q2
models[[i]] <- plm(
    formula = regressors[[1]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

## Model 3
i = 3
data = panel_time_q3
models[[i]] <- plm(
    formula = regressors[[1]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

## Model 4
i = 4
data = panel_time_q4
models[[i]] <- plm(
    formula = regressors[[1]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

In [280]:
# Create a sequence of years from 1986 to 2023
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Use the list in the stargazer function
stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]],  # Include all models
    type = "latex", # Output as LaTeX
    se = lapply(vcov_list, function(x) sqrt(diag(x))), # Extract robust SEs
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE, # Do not include default dependent variable labels
    dep.var.caption = "", # Remove the "Dependent variable:" text
    #column.labels = c("log(Water Area)", "log(Fire Area)"), # Specify dependent variables above model numbers
    #column.separate = c(2, 2), # Group columns by dependent variable
    omit.stat = c("f", "ser", "rsq"), # Omit F-stat and SER if needed
    multicolumn = TRUE,
    add.lines = list(
        c('Subset', "1985-1994", "1995-2004", "2005-2014", "2015-2023"),
        c("Cluster", "Level 5", "Level 5", "Level 5", "Level 5"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "Yes", "Yes", "Yes", "Yes")
    ),
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    flip = TRUE,
    notes = "Standard errors are shown in parentheses.", # Add notes
    out = "output_table.tex" # Specify the output file path
)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Feb 05, 2025 - 11:50:44
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)} & \multicolumn{1}{c}{(3)} & \multicolumn{1}{c}{(4)}\\ 
\hline \\[-1.8ex] 
 (Intercept) & -0.061^{***} & -0.038^{***} & 0.037^{***} & -0.021^{***} \\ 
  & (0.005) & (0.002) & (0.003) & (0.002) \\ 
  log(Anthropic Upstream) & -0.360^{*} & -0.241^{**} & -1.274^{***} & -0.377^{***} \\ 
  & (0.194) & (0.103) & (0.364) & (0.093) \\ 
 \hline \\[-1.8ex] 
Subset & 1985-1994 & 1995-2004 & 2005-2014 & 2015-2023 \\ 
Cluster & Level 5 & Level 5 & Level 5 & Level 5 \\ 
Entity FE & Yes & Yes & Yes & Yes \\ 
Year Dummy & Yes & Yes & Yes & Yes \\ 
Observations & \mult

### 7.3.2 Fire

In [281]:
## Model 1
i = 1
data = panel_time_q1
models[[i]] <- plm(
    formula = regressors[[2]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

## Model 2
i = 2
data = panel_time_q2
models[[i]] <- plm(
    formula = regressors[[2]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

## Model 3
i = 3
data = panel_time_q3
models[[i]] <- plm(
    formula = regressors[[2]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

## Model 4
i = 4
data = panel_time_q4
models[[i]] <- plm(
    formula = regressors[[2]], data = data, 
    index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
)
vcov_list[[i]] <- plm::vcovHC(
    models[[i]], method = "arellano", type = "HC1",
    cluster = "group", group = data$level5
)

In [282]:
# Create a sequence of years from 1986 to 2023
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Use the list in the stargazer function
stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]],  # Include all models
    type = "latex", # Output as LaTeX
    se = lapply(vcov_list, function(x) sqrt(diag(x))), # Extract robust SEs
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE, # Do not include default dependent variable labels
    dep.var.caption = "", # Remove the "Dependent variable:" text
    #column.labels = c("log(Water Area)", "log(Fire Area)"), # Specify dependent variables above model numbers
    #column.separate = c(2, 2), # Group columns by dependent variable
    omit.stat = c("f", "ser", "rsq"), # Omit F-stat and SER if needed
    multicolumn = TRUE,
    add.lines = list(
        c('Subset', "1985-1994", "1995-2004", "2005-2014", "2015-2023"),
        c("Cluster", "Level 5", "Level 5", "Level 5", "Level 5"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "Yes", "Yes", "Yes", "Yes")
    ),
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    flip = TRUE,
    notes = "Standard errors are shown in parentheses.", # Add notes
    out = "output_table.tex" # Specify the output file path
)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Feb 05, 2025 - 11:50:45
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)} & \multicolumn{1}{c}{(3)} & \multicolumn{1}{c}{(4)}\\ 
\hline \\[-1.8ex] 
 (Intercept) & -0.058^{***} & -0.018^{***} & -0.094^{***} & -0.057^{***} \\ 
  & (0.005) & (0.003) & (0.004) & (0.004) \\ 
  log(Anthropic Upstream) & 0.620^{***} & 0.296^{***} & 0.477 & 0.738^{***} \\ 
  & (0.173) & (0.081) & (0.364) & (0.221) \\ 
 \hline \\[-1.8ex] 
Subset & 1985-1994 & 1995-2004 & 2005-2014 & 2015-2023 \\ 
Cluster & Level 5 & Level 5 & Level 5 & Level 5 \\ 
Entity FE & Yes & Yes & Yes & Yes \\ 
Year Dummy & Yes & Yes & Yes & Yes \\ 
Observations & \multicolum

## 7.3 Most Relevant Level 3 Basins

In [283]:
panel <- panel %>% mutate(level3 = substr(wts_cd_pfafstetterbasin, 1, 3))
panel <- panel %>% mutate(level3 = lapply(level3, function(x) as.double(x)))
panel$level3 <- unlist(panel$level3)
panel <- panel %>% select(1:4, level3, everything())

mutate: converted 'level3' from double to character (0 new NA)
mutate: converted 'level3' from character to list (0 new NA)
select: no changes


In [284]:
list_level3 <- panel$level3 %>% unique()
no_unique_level3 <- length(list_level3)
no_unique_level3

[1] 8

In [285]:
list_level3

[1] 891 893 894 895 896 897 898 899

In [286]:
# Assuming panel is your main dataset and list_level4 contains the level4 codes
for (level3_code in list_level3) {
    subset_panel <- panel[panel$level3 == level3_code, ]
    dataset_name <- paste0("panel_", level3_code)
    assign(dataset_name, subset_panel)
}

In [287]:
colnames(panel_891)

[1] "wts_cd_pfafstetterbasin"     "year"                       
 [3] "level5"                      "level4"                     
 [5] "level3"                      "water_basin_km2"            
 [7] "wetland_basin_km2"           "fire_basin_km2"             
 [9] "antropic_upstream_km2"       "antropic_downstream_km2"    
[11] "area_basin_km2"              "area_upstream_km2"          
[13] "area_downstream_km2"         "water_share"                
[15] "wetland_share"               "fire_share"                 
[17] "antropic_upstream_share"     "antropic_downstream_share"  
[19] "upstream_rainfall"           "in_biome"                   
[21] "closed_basin"                "log_water_basin_km2"        
[23] "log_wetland_basin_km2"       "log_fire_basin_km2"         
[25] "log_antropic_upstream_km2"   "log_antropic_downstream_km2"
[27] "log_upstream_rainfall"

In [288]:
# 1. Set the multiple formulas
regressors <- list(
  formula_water = log_water_basin_km2 ~ log_antropic_upstream_km2 + factor(year),
  formula_fire = log_fire_basin_km2 ~ log_antropic_upstream_km2 + factor(year)
)

# 2. Lists to append the objects
models <- list()
vcov_list <- list()

In [289]:
# Assuming list_level3 contains the numbers 891 to 899
list_level3

# Initialize lists to store models and vcov matrices
models <- list()
vcov_list <- list()

# Loop through each number in list_level3
for (i in seq_along(list_level3)) {
    # Construct the dataset name
    dataset_name <- paste0("panel_", list_level3[i])
    
    # Get the dataset
    data <- get(dataset_name)
    
    # Estimate the model
    models[[i]] <- plm(
        formula = regressors[[1]], data = data, 
        index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
    )
    
    # Estimate the vcov matrix
    vcov_list[[i]] <- plm::vcovHC(
        models[[i]], method = "arellano", type = "HC1",
        cluster = "group", group = data$level5
    )
}

[1] 891 893 894 895 896 897 898 899

In [290]:
# Years to omit
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Check standard errors
se_list <- lapply(vcov_list[1:8], function(x) sqrt(diag(x)))

# Simplified stargazer call
table <- stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]], 
    type = "text",
    se = se_list,
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE,
    dep.var.caption = "",
    omit.stat = c("f", "ser", "rsq"),
    multicolumn = TRUE,
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    flip = TRUE,
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    notes = "Standard errors are shown in parentheses.",
    out = "output_table.tex"
)


                            (1)         (2)        (3)        (4)   
--------------------------------------------------------------------
(Intercept)              -0.050***    -0.005*   -0.065***  -0.032***
                          (0.009)     (0.003)    (0.010)    (0.002) 
log(Anthropic Upstream)    -1.439               -0.815***  -0.227** 
                          (0.972)                (0.239)    (0.109) 
--------------------------------------------------------------------
Observations               1,178        266       1,672     16,074  
Adjusted R2                0.407       0.356      0.428      0.207  
Note:                                    *p<0.1; **p<0.05; ***p<0.01
                           Standard errors are shown in parentheses.


In [291]:
# Years to omit
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Check standard errors
se_list <- lapply(vcov_list[1:8], function(x) sqrt(diag(x)))

# Simplified stargazer call
table <- stargazer(
    models[[5]], models[[6]], models[[7]], models[[8]], 
    type = "text",
    se = se_list,
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE,
    dep.var.caption = "",
    omit.stat = c("f", "ser", "rsq"),
    multicolumn = TRUE,
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    flip = TRUE,
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    notes = "Standard errors are shown in parentheses.",
    out = "output_table.tex"
)


                            (1)        (2)        (3)        (4)    
--------------------------------------------------------------------
(Intercept)               -0.009    -0.032***  -0.037***  -0.014*** 
                          (0.009)    (0.003)    (0.010)    (0.002)  
log(Anthropic Upstream)   -0.140      0.059      0.069      -0.095  
                          (0.972)               (0.239)    (0.109)  
--------------------------------------------------------------------
Observations              23,598      3,534      3,762      21,090  
Adjusted R2                0.071      0.214      0.261      0.091   
Note:                                    *p<0.1; **p<0.05; ***p<0.01
                           Standard errors are shown in parentheses.


In [292]:
# Assuming list_level3 contains the numbers 891 to 899
list_level3

# Initialize lists to store models and vcov matrices
models <- list()
vcov_list <- list()

# Loop through each number in list_level3
for (i in seq_along(list_level3)) {
    # Construct the dataset name
    dataset_name <- paste0("panel_", list_level3[i])
    
    # Get the dataset
    data <- get(dataset_name)
    
    # Estimate the model
    models[[i]] <- plm(
        formula = regressors[[2]], data = data, 
        index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
    )
    
    # Estimate the vcov matrix
    vcov_list[[i]] <- plm::vcovHC(
        models[[i]], method = "arellano", type = "HC1",
        cluster = "group", group = data$level5
    )
}

[1] 891 893 894 895 896 897 898 899

In [293]:
# Years to omit
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Check standard errors
se_list <- lapply(vcov_list[1:8], function(x) sqrt(diag(x)))

# Simplified stargazer call
table <- stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]], 
    type = "text",
    se = se_list,
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE,
    dep.var.caption = "",
    omit.stat = c("f", "ser", "rsq"),
    multicolumn = TRUE,
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    flip = TRUE,
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    notes = "Standard errors are shown in parentheses.",
    out = "output_table.tex"
)

Warning message in sqrt(diag(x)):
"NaNs produced"



                             (1)          (2)       (3)       (4)   
--------------------------------------------------------------------
(Intercept)               -0.046***      0.000    -0.012   -0.035***
                           (0.013)                (0.007)   (0.003) 
log(Anthropic Upstream)    5.994**                0.720**    0.312  
                           (2.611)                (0.307)   (0.191) 
--------------------------------------------------------------------
Observations                1,178         266      1,672    16,074  
Adjusted R2                 0.448        0.595     0.401     0.202  
Note:                                    *p<0.1; **p<0.05; ***p<0.01
                           Standard errors are shown in parentheses.


In [294]:
# Years to omit
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Check standard errors
se_list <- lapply(vcov_list[1:8], function(x) sqrt(diag(x)))

# Simplified stargazer call
table <- stargazer(
    models[[5]], models[[6]], models[[7]], models[[8]], 
    type = "text",
    se = se_list,
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE,
    dep.var.caption = "",
    omit.stat = c("f", "ser", "rsq"),
    multicolumn = TRUE,
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    flip = TRUE,
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    notes = "Standard errors are shown in parentheses.",
    out = "output_table.tex"
)

Warning message in sqrt(diag(x)):
"NaNs produced"



                             (1)          (2)       (3)       (4)   
--------------------------------------------------------------------
(Intercept)               -0.038***      0.011   -0.018**  -0.039***
                           (0.013)                (0.007)   (0.003) 
log(Anthropic Upstream)     0.366        0.040    -0.191   0.743*** 
                           (2.611)                (0.307)   (0.191) 
--------------------------------------------------------------------
Observations                23,598       3,534     3,762    21,090  
Adjusted R2                 0.205        0.357     0.184     0.158  
Note:                                    *p<0.1; **p<0.05; ***p<0.01
                           Standard errors are shown in parentheses.


In [295]:
1873/15

[1] 124.8667

## 7.4 Groups of Basins

In [296]:
panel_path <- "output/data/panel_group.csv" #Path to the panel data
panel_group.R <- read_csv(panel_path)
panel_group <- panel_group.R %>% tibble()
panel_group <- panel_group %>% mutate(
    log_water_basin_km2 = asinh(water_basin_km2),
    log_wetland_basin_km2 = asinh(wetland_basin_km2),
    log_fire_basin_km2 = asinh(fire_basin_km2),
    log_antropic_upstream_km2 = asinh(antropic_upstream_km2), 
    log_antropic_downstream_km2 = asinh(antropic_downstream_km2),
    log_upstream_rainfall = asinh(upstream_rainfall)
    )
panel_group <- data.frame(panel_group)
panel_group

Rows: 73047 Columns: 22
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (1): group
dbl (21): wts_cd_pfafstetterbasin, year, level5, level4, level3, water_basin...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
mutate: new variable 'log_water_basin_km2' (double) with 17,215 unique values and 0% NA
        new variable 'log_wetland_basin_km2' (double) with 27,802 unique values and 0% NA
        new variable 'log_fire_basin_km2' (double) with 16,376 unique values and 0% NA
        new variable 'log_antropic_upstream_km2' (double) with 27,429 unique values and 0% NA
        new variable 'log_antropic_downstream_km2' (double) with 22,731 unique values and 0% NA
        new variable 'log_upstream_rainfall' (double) with 32,990 unique values and 0% NA


wts_cd_pfafstetterbasin,year,level5,level4,level3,water_basin_km2,wetland_basin_km2,fire_basin_km2,antropic_upstream_km2,antropic_downstream_km2,...,upstream_rainfall,in_biome,closed_basin,group,log_water_basin_km2,log_wetland_basin_km2,log_fire_basin_km2,log_antropic_upstream_km2,log_antropic_downstream_km2,log_upstream_rainfall
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
891782,1985,89178,8917,891,8.4870,195.2496,6.7707,0,0,...,0,0,1,G1,2.835136,5.967432,2.6111611,0,0,0
891782,1986,89178,8917,891,8.8191,181.3527,20.3463,0,0,...,0,0,1,G1,2.873266,5.893599,3.7066496,0,0,0
891782,1987,89178,8917,891,9.2313,171.3231,4.4928,0,0,...,0,0,1,G1,2.918668,5.836707,2.2077846,0,0,0
891782,1988,89178,8917,891,9.0243,172.9323,55.7037,0,0,...,0,0,1,G1,2.896124,5.846056,4.7132743,0,0,0
891782,1989,89178,8917,891,8.8857,172.6758,16.6122,0,0,...,0,0,1,G1,2.880742,5.844571,3.5041892,0,0,0
891782,1990,89178,8917,891,8.5599,168.1218,12.1068,0,0,...,0,0,1,G1,2.843630,5.817845,3.1886157,0,0,0
891782,1991,89178,8917,891,8.5896,164.2545,4.1067,0,0,...,0,0,1,G1,2.847071,5.794574,2.1202714,0,0,0
891782,1992,89178,8917,891,9.4059,163.0755,4.7988,0,0,...,0,0,1,G1,2.937298,5.787370,2.2721966,0,0,0
891782,1993,89178,8917,891,9.4734,161.5419,38.7567,0,0,...,0,0,1,G1,2.944409,5.777921,4.3506172,0,0,0


In [297]:
groups_list <- panel_group %>% distinct(group) %>% pull(group)
groups_list

distinct: removed 73,042 rows (>99%), 5 rows remaining


[1] "G1" "G2" "G3" "G4" "G5"

In [298]:
# Create multiple subsets with datasets for each group
groups_list <- panel_group %>% distinct(group) %>% pull(group)
for (group in groups_list) {
    subset_group <- panel_group[panel_group$group == group, ]
    dataset_name <- paste0("panel_", group)
    assign(dataset_name, subset_group)
}

distinct: removed 73,042 rows (>99%), 5 rows remaining


In [299]:
groups_list

[1] "G1" "G2" "G3" "G4" "G5"

In [300]:
# Initialize lists to store models and vcov matrices
models <- list()
vcov_list <- list()

# Loop through each number in list_level3
for (i in seq_along(groups_list)) {
    # Construct the dataset name
    dataset_name <- paste0("panel_", groups_list[i])
    
    # Get the dataset
    data <- get(dataset_name)
    
    # Estimate the model
    models[[i]] <- plm(
        formula = regressors[[1]], data = data, 
        index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
    )
    
    # Estimate the vcov matrix
    vcov_list[[i]] <- plm::vcovHC(
        models[[i]], method = "arellano", type = "HC1",
        cluster = "group", group = data$level5
    )
}

In [301]:
# Years to omit
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Check standard errors
se_list <- lapply(vcov_list[1:5], function(x) sqrt(diag(x)))

# Simplified stargazer call
table <- stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]], models[[5]], 
    type = "latex",
    se = se_list,
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE,
    dep.var.caption = "",
    omit.stat = c("f", "ser", "rsq"),
    multicolumn = TRUE,
    column.labels = c('Group 1','Group 2','Group 3','Group 4','Group 5'),
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    add.lines = list(
        c("Cluster", "Level 5", "Level 5", "Level 5", "Level 5", "Level 5"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "Yes", "Yes", "Yes", "Yes", "Yes")
    ),
    flip = TRUE,
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    notes = "Standard errors are shown in parentheses.",
    out = "output_table.tex"
)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Feb 05, 2025 - 11:50:54
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{Group 1} & \multicolumn{1}{c}{Group 2} & \multicolumn{1}{c}{Group 3} & \multicolumn{1}{c}{Group 4} & \multicolumn{1}{c}{Group 5} \\ 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)} & \multicolumn{1}{c}{(3)} & \multicolumn{1}{c}{(4)} & \multicolumn{1}{c}{(5)}\\ 
\hline \\[-1.8ex] 
 (Intercept) & -0.005^{*} & -0.039^{***} & -0.035^{***} & -0.013^{***} & -0.016^{***} \\ 
  & (0.003) & (0.004) & (0.003) & (0.001) & (0.002) \\ 
  log(Anthropic Upstream) & -0.486 & -1.103^{***} & -0.230^{*} & -0.073 & -0.164 \\ 
  & (0.370) & (0.202) & (0.121) & (0.064) & (0.124) \\ 
 \h

In [302]:
# Initialize lists to store models and vcov matrices
models <- list()
vcov_list <- list()

# Loop through each number in list_level3
for (i in seq_along(groups_list)) {
    # Construct the dataset name
    dataset_name <- paste0("panel_", groups_list[i])
    
    # Get the dataset
    data <- get(dataset_name)
    
    # Estimate the model
    models[[i]] <- plm(
        formula = regressors[[2]], data = data, 
        index = c("wts_cd_pfafstetterbasin", "year"), model = "fd"
    )
    
    # Estimate the vcov matrix
    vcov_list[[i]] <- plm::vcovHC(
        models[[i]], method = "arellano", type = "HC1",
        cluster = "group", group = data$level5
    )
}

In [303]:
# Years to omit
years <- 1986:2023
years_to_omit <- paste0("factor\\(year\\)", years)

# Check standard errors
se_list <- lapply(vcov_list[1:5], function(x) sqrt(diag(x)))

# Simplified stargazer call
table <- stargazer(
    models[[1]], models[[2]], models[[3]], models[[4]], models[[5]], 
    type = "latex",
    se = se_list,
    align = TRUE,
    digits = 3,
    no.space = TRUE,
    dep.var.labels.include = FALSE,
    dep.var.caption = "",
    omit.stat = c("f", "ser", "rsq"),
    multicolumn = TRUE,
    column.labels = c('Group 1','Group 2','Group 3','Group 4','Group 5'),
    covariate.labels = c("(Intercept)", "log(Anthropic Upstream)"),
    intercept.bottom = FALSE,
    add.lines = list(
        c("Cluster", "Level 5", "Level 5", "Level 5", "Level 5", "Level 5"), # Indicate clustering
        c("Entity FE", "Yes", "Yes", "Yes", "Yes", "Yes"), # Add fixed effects line
        c('Year Dummy', "Yes", "Yes", "Yes", "Yes", "Yes")
    ),
    flip = TRUE,
    omit = years_to_omit, # Specify the rows to omit based on coefficient names
    notes = "Standard errors are shown in parentheses.",
    out = "output_table.tex"
)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Feb 05, 2025 - 11:50:56
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{Group 1} & \multicolumn{1}{c}{Group 2} & \multicolumn{1}{c}{Group 3} & \multicolumn{1}{c}{Group 4} & \multicolumn{1}{c}{Group 5} \\ 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)} & \multicolumn{1}{c}{(3)} & \multicolumn{1}{c}{(4)} & \multicolumn{1}{c}{(5)}\\ 
\hline \\[-1.8ex] 
 (Intercept) & -0.050^{***} & -0.018^{***} & -0.050^{***} & -0.031^{***} & -0.037^{***} \\ 
  & (0.006) & (0.003) & (0.005) & (0.002) & (0.002) \\ 
  log(Anthropic Upstream) & -0.399 & 0.868^{***} & 0.311 & 0.339^{***} & 0.664^{***} \\ 
  & (1.188) & (0.248) & (0.227) & (0.078) & (0.152) \